In [1]:
import re
import time
from datetime import datetime, timedelta
import math
from operator import itemgetter


In [2]:
date2report = dict()
date2vector = dict()
date2change = dict()
percentage_change = dict()
idf = dict()
years = ["2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]
pattern = re.compile(r'^(\d\d\.\d\d\.\d\d\d\d)$')
punctuations = open("punctuations.txt").read()
full_text = []

expected_sign = dict()
expected_num = dict()
expected_num_dist = dict()
observed = dict()


In [78]:
# WORKS FINE
def fill_percentage_change():
    with open("percentage_change_cumm.txt", "r+") as file:
        lines = file.readlines()
        for line in lines:
            (date, change) = tuple(line.strip().split('$'))
            percentage_change[date] = float(change)
 

def fill_observed_percentage_change_2019():
    with open("percentage_change_2019.txt", "r+") as file:
        lines = file.readlines()
        for line in lines:
            (date, change) = tuple(line.strip().split("$"))
            observed[date] = float(change)
     
    
def get_next_date(date_time_obj):
    date_time_obj += timedelta(days=1)
    next_day = date_time_obj.strftime('%d.%m.%Y')
    while next_day not in price_changes:
        date_time_obj += timedelta(days=1)
        next_day = date_time_obj.strftime('%d.%m.%Y')    
    return next_day


def fill_date2report(year):
    key = ""
    pattern = re.compile(r'^(\d\d\.\d\d\.\d\d\d\d)$')

    with open("reports_"+year+".txt", "r+") as file:
        lines = [line.strip() for line in file.readlines()]
        for line in lines:
            line = line.rstrip('\n')
            if year is not "2019":
                full_text.extend(line.split())
            if line == "":
                continue
            if re.search(pattern, line):
                new_entry = True
                key = line.replace('\n', "")
            else:
                new_entry = False                
            if new_entry:
                date2report[key] = ""
            else:
                date2report[key] +=line

                
def fill_date2change():
    for date in date2report:
        if date == '28.12.2018':
            continue
        date_time_obj = datetime.strptime(date, '%d.%m.%Y')
        date2change[date] = percentage_change[get_next_date(date_time_obj)]
        
        
def preprocess_text(text):
    text = text.lower()
    for punc in punctuations:
        text = text.replace(punc, ' ')
    return text


def generate_vector_space(text):
    text = preprocess_text(text)
    tokens = text.split()
    tf = {i:tokens.count(i) for i in set(tokens)}
    vector = {}
    for term in tf:
        if term in full_text:
            vector[term] = tf[term]
            temp = len(date2report)/idf[term]
            vector[term] *= math.log(temp)
        else:
            vector[term] = 0
    return vector


def fill_date2vector():
    for date in date2report:
        date2report[date] = preprocess_text(date2report[date])
        print(date2report[date])
        tokens = date2report[date].split()
        tokens = list(set(tokens))
        for token in tokens:
            if token in idf:
                idf[token] += 1
            else:
                idf[token] = 1
    
    for date in date2report:
        date2vector[date] = generate_vector_space(date2report[date])
        

def length_vector(vector):
    total = 0
    for dim in vector:
        total += vector[dim]*vector[dim]
    return math.sqrt(total)

def dot_product(vector1, vector2):
    product = 0
    for dim in vector1:
        if dim in vector2:
            product += vector1[dim] * vector2[dim]
    return product

def cosine_similarity(vector1, vector2):
    return dot_product(vector1, vector2) / (length_vector(vector1)*length_vector(vector2))
    

In [10]:
def fill_train_data():
    fill_percentage_change()
    for year in years:
        fill_date2report(year)
    fill_date2change()
    fill_date2vector()
    

In [13]:
# returns suggested other documents with their cosine similarity scores in descending order
def cosine_similarity_scores_for_new_document(document):
    scores = []
    vector = generate_vector_space(document)
    for date in date2report:
        score = cosine_similarity(vector, date2vector[date])
        scores.append((date, score))
    scores.sort(key=itemgetter(1), reverse=True)
    return scores


def get_test_reports(year):
    key = ""
    pattern = re.compile(r'^(\d\d\.\d\d\.\d\d\d\d)$')
    test_data = dict()
    with open("reports_"+year+".txt", "r+") as file:
        lines = [line.strip() for line in file.readlines()]
        for line in lines:
            line = line.rstrip('\n')
            if line == "":
                continue
            if re.search(pattern, line):
                new_entry = True
                key = line.replace('\n', "")
            else:
                new_entry = False                
            if new_entry:
                test_data[key] = ""
            else:
                test_data[key] +=line
    return test_data




In [ ]:
# TODO: 
# [x] 2019 rapor datasini oku
# [x] 2019 change datasini store et
# [x] 2019 vectorlerini generate et
# [x] 2019 vectorlerinin her biri icin diger haberlerin vektorleri ile cosine similarty kiyaslamasi yap
# [x] cosine score'larini en yuksek'ten en dusuge sirala
# [x] 3 cesit KNN uygula bu siralamalara gore (sadece sign'a gore, artis-azalis degerlerine gore, artis-azalis ve uzaklik degerlerine gore)
# [x] k=3, k=5 ve k=7 icin dene
# [x] accuracy hesaplamasi yap


fill_data2report("2019")
test_data = data2report["2019"]
scores = []
for data in test_data:
    vector = generate_vector_space(test_data[data])
    
    
    
    


In [15]:
# KNNs

# returns the calculated sign (True => artis, False => dusus) for the given scores value with given k
def KNN_sign(k, scores):
    num_pos = 0
    num_neg = 0
    for score in scores[0:k]:
        if score[1] > 0:
            num_pos += 1
        else:
            num_neg += 1
    return 1 if num_pos>num_neg else -1


# returns the arithmetic mean of the k nearest neighbors
def KNN_number(k, scores):
    total = 0
    for score in scores[0:k]:
        total += score[1]
    return total/k


# returns upwards or downwards movement according to the weighted average of k nearest neighbors parametrized by their cosine similarity
def KNN_number_distance(k, scores):
    total = 0
    for score in scores[0:k]:
        total += score[1] * date2change[score[0]]
    return 1 if total>0 else -1


def acquire_KNN_suggestions(date, document, k):
    scores = cosine_similarity_scores_for_new_document(document)
    print(date)
    print(scores[0:k])
    sign = KNN_sign(k, scores)
    num_dist = KNN_number_distance(k, scores)
    num = KNN_number(k, scores)
    return sign, num_dist, num



In [18]:
def calculate_accuracy_sign(expected_sign, observed):
    accurate = 0 
    for date in expected_sign:
        factor = expected_sign[date] * observed[date]
        if factor>0:
            accurate += 1
    return accurate/len(expected_sign)
    
    
def chi_square(expected_num, observed):
    chi_square_score = 0
    for date in expected_num:
        o = observed[date]
        e = expected_num[date]
        chi_square_score += (o-e)*(o-e) / e
    return chi_square_score
        

def generate_all_KNN_results_fill_expected(k):
    expected_sign.clear()
    expected_num.clear()
    expected_num_dist.clear()
    
    print("expected's renewed")
    
    for date in reports_2019:
        document = reports_2019[date]
        scores = cosine_similarity_scores_for_new_document(document)
        print("cosine similarity scores for doc in "+date+" calculated")
        sign = KNN_sign(k, scores)
        print("\tsign estimation: done")
        num = KNN_number(k, scores)
        print("\tvalue estimation: done")
        num_dist = KNN_number_distance(k, scores)
        print("\tsign estimation with distance: done")
        expected_sign[date] = sign
        expected_num[date] = num
        expected_num_dist[date] = num_dist
    return expected_sign, expected_num, expected_num_dist
    
    
def calculate_accuracy(k):
    expected_sign, expected_num, expected_num_dist = generate_all_KNN_results_fill_expected(k)
    print("All KNN results generated")
    a1 = calculate_accuracy_sign(expected_sign, observed)
    print("sign accuracy calculated")
    a2 = chi_square(expected_num, observed)
    print("Chi-Square calculated")
    a3 = calculate_accuracy_sign(expected_num_dist, observed)
    print("sign accuracy by distance calculated")
    return a1,a2,a3    
    

In [24]:
def generate_all_KNN_results(k):    
    for date in reports_2019:
        report = reports_2019[date]
        vector = generate_vector_space(report)
        cosine_scores[date] = cosine_similarity_scores_for_new_document(report)
        sign, num_dist, num = acquire_KNN_suggestions(date, report, 5)
        expected_sign[date] = sign
        expected_num_dist[date] = num_dist
        expected_num[date] = num
    print(calculate_accuracy())
    

In [12]:
from operator import itemgetter
cosine_scores = dict()

def fill_test_data()
    fill_observed_percentage_change_2019()
    reports_2019 = get_test_reports("2019")
    generate_all_KNN_results_fill_expected(5)



In [28]:
calculate_accuracy(5)


expected's renewed
cosine similarity scores for doc in 09.05.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 03.05.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 25.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 17.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 12.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 21.03.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 12.03.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with di

(0.2857142857142857, 133.00894967043124, 0.14285714285714285)

In [30]:
calculate_accuracy(3)


expected's renewed
cosine similarity scores for doc in 09.05.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 03.05.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 25.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 17.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 12.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 21.03.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 12.03.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with di

(0.2857142857142857, 127.68321509850091, 0.21428571428571427)

In [31]:
calculate_accuracy(7)


expected's renewed
cosine similarity scores for doc in 09.05.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 03.05.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 25.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 17.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 12.04.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 21.03.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with distance: done
cosine similarity scores for doc in 12.03.2019 calculated
	sign estimation: done
	value estimation: done
	sign estimation with di

(0.2857142857142857, 137.53284313228372, 0.21428571428571427)

In [27]:
document = "The Incorporation has decided to start operating scheduled flights, based on aircraft availability, to Zonguldak Airport in Turkey."
sign, num_dict, num = acquire_KNN_suggestions('03.05.2019', document, 3)
print(sign)
print(num_dict)
print(num)


03.05.2019
[('27.09.2013', 0.856361965552941), ('27.01.2014', 0.856361965552941), ('15.11.2013', 0.855505643827547)]
1
1
0.8560765249778096


In [32]:
print(date2report['27.09.2013'])


the incorporation has decided to start operating scheduled flights  based on aircraft availability to lahore in pakistan 


In [33]:
print(date2report['27.01.2014'])


the incorporation has decided to start operating scheduled flights  based on aircraft availability to pisa in italy 


In [34]:
print(expected_sign)

{'09.05.2019': 1, '03.05.2019': 1, '25.04.2019': 1, '17.04.2019': 1, '12.04.2019': 1, '21.03.2019': 1, '12.03.2019': 1, '05.03.2019': 1, '18.02.2019': 1, '14.02.2019': 1, '12.02.2019': 1, '14.01.2019': 1, '10.01.2019': 1, '04.01.2019': 1}


In [35]:
print(expected_num)

{'09.05.2019': 0.5931406844712681, '03.05.2019': 0.8042125284527103, '25.04.2019': 0.8148090177048406, '17.04.2019': 0.49732474941970567, '12.04.2019': 0.9154709437278489, '21.03.2019': 0.21905455983823482, '12.03.2019': 0.9469798492357305, '05.03.2019': 0.9035439059761717, '18.02.2019': 0.8136493522737709, '14.02.2019': 0.2798879146164436, '12.02.2019': 0.9087283275239108, '14.01.2019': 0.6539974796429396, '10.01.2019': 0.9654466693615688, '04.01.2019': 0.3864305236889329}


In [38]:
acquire_KNN_suggestions('03.05.2019', document, 5)

03.05.2019
[('27.09.2013', 0.856361965552941), ('27.01.2014', 0.856361965552941), ('15.11.2013', 0.855505643827547), ('09.02.2015', 0.7897303769074531), ('12.10.2015', 0.7886895131308796)]


(1, 1, 0.8293298929943523)

In [39]:
print(date2vector)

{'23.12.2011': {'start': 3.1540689767588823, 'currently': 4.712528704308591, 'aalborg': 0, 'aircraft': 4.965206227610183, 'and': 1.4373724867649222, 'has': 1.6192646869096456, 'incorporation': 3.577231412206289, 'bremen': 0, 'i̇stanbul': 0, 'be': 1.2261735143061285, 'open': 11.911710595681937, 'availability': 11.76596034759985, 'lagos': 0, 'in': 1.953029968522073, 'extend': 4.712528704308591, 'kinshasa': 0, 'aviation': 5.9558552978409685, 'civil': 7.2278328312809625, 'authorizations': 18.729527439332855, 'the': 0.8270980234534037, 'billund': 0, 'on': 5.8491545161513345, 'based': 10.008037599511006, 'accra': 0, 'obtaining': 18.145856176759498, 'as': 1.871265561407836, 'operating': 4.864458752850824, 'to': 1.1105626889563496, 'will': 2.7606483882667736, 'year': 10.134756795292198, 'decided': 5.346960402891453, 'leipzig': 0, 'route': 23.366153880644287, 'of': 0.1727863236429546, 'upon': 16.20243402057154, 'abidjan': 0, 'istanbul': 0, 'edinburgh': 0, 'january': 0}, '20.12.2011': {'cost': 3

In [41]:
all_data = dict()
years.append("2019")

for year in years:
    key = ""
    pattern = re.compile(r'^(\d\d\.\d\d\.\d\d\d\d)$')
    with open("reports_"+year+".txt", "r+") as file:
        lines = [line.strip() for line in file.readlines()]
        for line in lines:
            line = line.rstrip('\n')
            if line == "":
                continue
            if re.search(pattern, line):
                new_entry = True
                key = line.replace('\n', "")
            else:
                new_entry = False                
            if new_entry:
                all_data[key] = ""
            else:
                all_data[key] +=line


print(all_data)




{'23.12.2011': 'The Incorporation has decided to start Istanbul-Kinshasa-İstanbul route, based on aircraft availability and upon obtaining the Civil Aviation authorizations.The Incorporation has decided to open Istanbul-Bremen-Istanbul route in the year 2012, based on aircraft availability and upon obtaining the authorizations.The Incorporation has decided to open Istanbul- Aalborg-Billund-İstanbul route in the year 2012, based on aircraft availability and upon obtaining the authorizations.The Incorporation has decided to start operating Istanbul- Edinburgh-İstanbul route in the year 2012, based on aircraft availability and upon obtaining the Civil Aviation authorizations.The Incorporation has decided to open Istanbul-Leipzig-Istanbul route in the year 2012, based on aircraft availability and upon obtaining the authorizations.The Incorporation has decided to open Istanbul-Accra-Abidjan-Istanbul route in the year 2012, based on aircraft availability and upon obtaining the authorizations

In [42]:

all_vectors = dict()

for entry in all_data:
    doc = all_data[entry]
    all_vectors[entry] = generate_vector_space(doc)

print(all_vectors)


{'23.12.2011': {'start': 3.1540689767588823, 'currently': 4.712528704308591, 'aalborg': 0, 'aircraft': 4.965206227610183, 'and': 1.4373724867649222, 'has': 1.6192646869096456, 'incorporation': 3.577231412206289, 'bremen': 0, 'i̇stanbul': 0, 'be': 1.2261735143061285, 'open': 11.911710595681937, 'availability': 11.76596034759985, 'lagos': 0, 'in': 1.953029968522073, 'extend': 4.712528704308591, 'kinshasa': 0, 'aviation': 5.9558552978409685, 'civil': 7.2278328312809625, 'authorizations': 18.729527439332855, 'the': 0.8270980234534037, 'billund': 0, 'on': 5.8491545161513345, 'based': 10.008037599511006, 'accra': 0, 'obtaining': 18.145856176759498, 'as': 1.871265561407836, 'operating': 4.864458752850824, 'to': 1.1105626889563496, 'will': 2.7606483882667736, 'year': 10.134756795292198, 'decided': 5.346960402891453, 'leipzig': 0, 'route': 23.366153880644287, 'of': 0.1727863236429546, 'upon': 16.20243402057154, 'abidjan': 0, 'istanbul': 0, 'edinburgh': 0, 'january': 0}, '20.12.2011': {'cost': 3

In [117]:
cosin_scores = dict()

for entry in all_vectors:
    cosin_scores[entry] = []
    v1 = all_vectors[entry]
    for vec in all_vectors:
        v2 = all_vectors[vec]
        if vec is entry:
            continue
        else:
            cosin_scores[entry].append((vec,cosine_similarity(v1, v2)))
    cosin_scores[entry].sort(key=itemgetter(1), reverse=True)
    cosin_scores[entry] = cosin_scores[entry][0:5]
    
print(cosin_scores)


{'23.12.2011': [('01.02.2011', 0.8590588011993397), ('02.03.2012', 0.8357397006928279), ('01.07.2011', 0.8311563005435904), ('18.11.2011', 0.8294158911787268), ('10.02.2012', 0.8154076338177827)], '20.12.2011': [('28.12.2012', 0.7571064587178497), ('14.01.2019', 0.4225569895927405), ('04.01.2018', 0.4072380253239536), ('25.08.2017', 0.39887397573802547), ('09.08.2018', 0.3536734686140062)], '12.12.2011': [('16.11.2011', 1.0), ('12.10.2011', 1.0), ('15.09.2011', 1.0), ('11.08.2011', 1.0), ('12.07.2011', 1.0)], '21.11.2011': [('15.04.2013', 0.6499738751259584), ('01.03.2013', 0.38061574002701304), ('21.01.2012', 0.2953008720771977), ('02.05.2014', 0.281877539583265), ('27.05.2014', 0.2659112617113111)], '18.11.2011': [('23.12.2011', 0.8294158911787268), ('19.10.2011', 0.8002362957851158), ('21.01.2011', 0.8002362957851158), ('10.02.2012', 0.771207484388781), ('18.05.2012', 0.7621046438980797)], '16.11.2011': [('12.12.2011', 1.0), ('12.10.2011', 1.0), ('15.09.2011', 1.0), ('11.08.2011', 1

In [118]:
price_changes = dict()

import csv



stock_file = "THYAO2011-2019.csv" #cmpe492
with open(stock_file, "rt") as infile:
    infile.readline()  
    read = csv.reader(infile, delimiter=",")
    for row in read :
        date = row[0]
        change = row[-1][0:-1]
        change = change.replace(',', '.')        
        price_changes[date]= float(change)


print(price_changes)






{'15.05.2019': -3.2, '14.05.2019': 1.13, '13.05.2019': 1.73, '10.05.2019': -2.49, '09.05.2019': -3.86, '08.05.2019': -4.14, '07.05.2019': -2.03, '06.05.2019': -0.72, '03.05.2019': -1.21, '02.05.2019': -1.68, '30.04.2019': 0.63, '29.04.2019': 2.16, '26.04.2019': 1.38, '25.04.2019': -2.21, '24.04.2019': 0.79, '22.04.2019': -1.55, '19.04.2019': -1.26, '18.04.2019': -0.14, '17.04.2019': 0.0, '16.04.2019': 2.94, '15.04.2019': 0.36, '12.04.2019': -1.0, '11.04.2019': 0.43, '10.04.2019': -3.39, '09.04.2019': 2.12, '08.04.2019': -2.68, '05.04.2019': 1.68, '04.04.2019': 6.79, '03.04.2019': 3.32, '02.04.2019': -0.23, '01.04.2019': 0.54, '29.03.2019': 2.62, '28.03.2019': -1.02, '27.03.2019': -5.56, '26.03.2019': -4.13, '25.03.2019': 4.23, '22.03.2019': -4.6, '21.03.2019': -0.56, '20.03.2019': 0.14, '19.03.2019': 1.65, '18.03.2019': 7.63, '15.03.2019': -0.38, '14.03.2019': 2.12, '13.03.2019': -2.6, '12.03.2019': 1.08, '11.03.2019': -1.89, '08.03.2019': -3.58, '07.03.2019': -2.91, '06.03.2019': -1.1

In [119]:
calculated_sign = dict()
num_calc_pos = 0
num_calc_neg = 0


for entry in cosin_scores:
    pos = 0
    for score in cosin_scores[entry]:
        date = score[0]
        date_time_obj = datetime.strptime(date, '%d.%m.%Y')
        
        if price_changes[get_next_date(date_time_obj)] > 0:
            pos += 1
        else:
            pos -= 1 
    if pos > 0:
        calculated_sign[entry] = 1
        num_calc_pos += 1
    else:
        calculated_sign[entry] = -1
        num_calc_neg += 1

print(calculated_sign)    

{'23.12.2011': -1, '20.12.2011': 1, '12.12.2011': -1, '21.11.2011': 1, '18.11.2011': 1, '16.11.2011': -1, '19.10.2011': 1, '14.10.2011': -1, '12.10.2011': -1, '23.09.2011': -1, '22.09.2011': 1, '15.09.2011': -1, '11.08.2011': -1, '05.08.2011': -1, '12.07.2011': -1, '05.07.2011': 1, '01.07.2011': -1, '06.06.2011': 1, '02.06.2011': 1, '01.06.2011': 1, '20.05.2011': -1, '06.05.2011': -1, '05.05.2011': -1, '27.04.2011': -1, '25.04.2011': -1, '22.04.2011': 1, '20.04.2011': 1, '14.04.2011': -1, '13.04.2011': 1, '08.04.2011': 1, '14.03.2011': -1, '11.03.2011': -1, '07.03.2011': 1, '03.03.2011': 1, '23.02.2011': 1, '18.02.2011': -1, '16.02.2011': 1, '15.02.2011': -1, '02.02.2011': 1, '01.02.2011': -1, '21.01.2011': 1, '18.01.2011': 1, '13.01.2011': -1, '04.01.2011': -1, '28.12.2012': 1, '14.12.2012': 1, '07.12.2012': 1, '23.11.2012': 1, '14.11.2012': 1, '09.11.2012': 1, '06.11.2012': -1, '22.10.2012': -1, '12.10.2012': 1, '10.10.2012': 1, '08.10.2012': -1, '05.10.2012': 1, '04.10.2012': 1, '17

In [120]:
accurate = 0
num_total_pos = 0
calc_pos_true = 0

num_total_neg = 0
calc_neg_true = 0


for sign in calculated_sign:
    date = sign
    date_time_obj = datetime.strptime(date, '%d.%m.%Y')
    if price_changes[get_next_date(date_time_obj)] > 0:
        num_total_pos += 1
        if calculated_sign[sign] > 0:
            calc_pos_true += 1
    else:
        num_total_neg += 1
        if calculated_sign[sign] < 0:
            calc_neg_true += 1



In [121]:
precision_pos = calc_pos_true / num_calc_pos
recall_pos = calc_pos_true / num_total_pos

print("Precision for positive sign: "+str(precision_pos))
print("Recall for positive sign: "+ str(recall_pos))

Precision for positive sign: 0.5287958115183246
Recall for positive sign: 0.5771428571428572


In [122]:
precision_neg = calc_neg_true / num_calc_neg
recall_neg = calc_neg_true / num_total_neg

print("Precision for negative sign: "+str(precision_neg))
print("Recall for negative sign: "+ str(recall_neg))

Precision for negative sign: 0.5286624203821656
Recall for negative sign: 0.4797687861271676


In [124]:
print(all_vectors['15.01.2014'])

{'our': 2.9346711422460334, 'has': 0.5397548956365485, 'show': 5.117993812416755, 'domestic': 2.784600770360205, 'especially': 5.811140992976701, 'cash': 3.865230843921387, 'working': 4.019381523748645, 'comments': 10.23598762483351, 'considered': 4.42484663185681, 'directors': 0, 'be': 1.2261735143061285, 'legislation': 4.712528704308591, 'corporations': 5.117993812416755, 'rates': 0, 'are': 3.1469011745368327, 'liabilities': 0, 'information': 7.730461687842774, 'sharp': 5.811140992976701, 'latest': 5.117993812416755, 'stated': 4.2017030805426, 'that': 3.1540689767588823, 'short': 5.811140992976701, 'increases': 2.9779276489204842, 'balance': 4.2017030805426, 'executed': 4.712528704308591, 'providing': 4.42484663185681, 'under': 3.3262343431887, 'influence': 5.117993812416755, 'constantly': 5.117993812416755, 'legal': 3.4132457201783297, 'airlines': 3.4335928615091995, 'of': 1.0367179418577277, 'numbers': 5.117993812416755, 'article': 3.2461916355151637, 'fx': 0, 'set': 4.424846631856

In [125]:
print(observed['15.01.2014'])

KeyError: '15.01.2014'